In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['facenet-keras', '5-celebrity-faces-dataset', 'face-model']


In [ ]:
# load the facenet model
facenet_model = load_model('../input/facenet-keras/facenet_keras.h5')
print('Loaded Model')

Loaded Model


/opt/conda/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:
from joblib import dump, load

In [ ]:
loaded_model = load('../input/face-model/facemodel.joblib')

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
## Load the trained LabelEncoder and SVM model
out_encoder=LabelEncoder()
loaded_encoder = out_encoder
loaded_predictor = loaded_model
loaded_facenet = facenet_model

In [ ]:
labels=['ben_afflek', 'elton_john', 'jerry_seinfeld', 'madonna',
       'mindy_kaling']

In [ ]:
loaded_encoder.fit(labels)

LabelEncoder()

In [ ]:
loaded_encoder.classes_

array(['ben_afflek', 'elton_john', 'jerry_seinfeld', 'madonna',
       'mindy_kaling'], dtype='<U14')

In [ ]:
# Function to get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = np.expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [ ]:
# Single Function capable to process input image and return the bounding box dimensions and the prediction
def face_recognize(image):
    ## Extract the face and bounding box dimensions from the image by using pretrained MTCNN model
    faces, box_dimensions = extract_face(image_pixels=image)
    X = np.asarray(faces)
    ## Get the Face Embeddings for the extracted face pixels and store as numpy array
    embedding = get_embedding(loaded_facenet, X)
    X = []
    X.append(embedding)
    X = np.asarray(X)
    ## Predict label for the face by using the pretrained models
    prediction = loaded_predictor.predict(X)
    predicted_label = loaded_encoder.inverse_transform([prediction])
    return predicted_label[0], box_dimensions

In [ ]:
def extract_face(filename=None, image_pixels=None, required_size=(160, 160)):
    if filename is not None:
        image = Image.open(filename)
        image = image.convert('RGB')
        pixels = np.asarray(image)
    elif image_pixels is not None:
        pixels = image_pixels
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    box_dimensions = (x1, y1, width, height)
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array, box_dimensions

In [ ]:
## Testing the model with the random image
filename = '../input/5-celebrity-faces-dataset/val/madonna/httpcdncdnjustjaredcomwpcontentuploadsheadlinesmadonnatalksparisattackstearsjpg.jpg'
image = Image.open(filename)
image = image.convert('RGB')
pixels = np.asarray(image)
face_recognize(pixels)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


('madonna', (98, 44, 95, 131))

In [ ]:
import tensorflow as tf


def load_model():
  model=tf.keras.models.load_model('/content/drive/MyDrive/fingerprint_model.h5')
  return model

model=load_model()


file = ("/content/3__M_Left_index_finger.BMP")
import cv2
from PIL import Image, ImageOps
import numpy as np

classes = ['ben_afflek','elton_john','jerry_seinfeld', 'madonna','mindy_kaling']

def import_and_predict(image_data, model):
    
        img_size=(224,224)
        #image = ImageOps.fit(image_data, img_size, Image.ANTIALIAS)
        img = np.expand_dims(image_data, axis=0)
    
        prediction = model.predict(img)
        
        return prediction


img = cv2.imread(file)     
img = cv2.resize(img,(224,224))
images_arr = np.asarray(img)
images_arr = images_arr.astype('float32')
images_arr = images_arr.reshape(-1, 224,224, 1)
predictions = import_and_predict(img, model)
score = tf.nn.softmax(predictions[0])
index = np.argmax(predictions[0])
predicted_class = classes[index]
probability = predictions[0][index]*100
print("This fingerprint belongs to ",predicted_class," with a ",probability," percent confidence.")


1/1 [==============================] - 2s 2s/step
This fingerprint belongs to  madonna  with a  100.0  percent confidence.
